In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import math



네이버 부동산 : https://land.naver.com/ 
참고 : https://cocoabba.tistory.com/58

In [5]:

keyword = "원하는 지역명"

url = "https://m.land.naver.com/search/result/{}".format(keyword)
res = requests.get(url)
res.raise_for_status()


soup = (str)(BeautifulSoup(res.text, "lxml"))
soup

'<!DOCTYPE HTML>\n<html lang="ko">\n<head>\n<link href="https://m.naver.com/favicon.ico" rel="shortcut icon" type="image/x-icon"/>\n<meta content="telephone=no" name="format-detection"/>\n<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>\n<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>\n<meta content="telephone=no" name="format-detection"/>\n<title>네이버페이 부동산</title>\n<meta content="text/javascript" http-equiv="Content-Script-Type"/>\n<meta content="text/css" http-equiv="Content-Style-Type"/>\n<meta contents="always" name="referrer"/>\n<link href="https://ssl.pstatic.net/static.land/static/space/css/deploy/20240502162214/min/space.min.css" rel="stylesheet" type="text/css"/><script src="https://ssl.pstatic.net/static.land/static/space/js/deploy/20240502162214/min/jquery_common.min.js" type="text/javascript"></script></head>\n<body>\n<div class="u_skip">\n<a href="#ct">컨텐츠 바로가기</a>\n</div>\n<

In [4]:
#응답 메시지 속에서 원하는 데이터 얻기
#  filter: {
#             lat: '37.550985',
#             lon: '126.849534',
#             z: '12',
#             cortarNo: '1150000000',
#             cortarNm: '강서구',
#             rletTpCds: '*',
#             tradTpCds: 'A1:B1:B2'
#         },

value = soup.split("filter: {")[1].split("}")[0].replace(" ","").replace("'","")

lat = value.split("lat:")[1].split(",")[0]
lon = value.split("lon:")[1].split(",")[0]
z = value.split("z:")[1].split(",")[0]
cortarNo = value.split("cortarNo:")[1].split(",")[0]
rletTpCds = value.split("rletTpCds:")[1].split(",")[0]
tradTpCds = value.split("tradTpCds:")[1].split()[0]

# lat - btm : 37.550985 - 37.4331698 = 0.1178152
# top - lat : 37.6686142 - 37.550985 = 0.1176292
lat_margin = 0.118

# lon - lft : 126.849534 - 126.7389841 = 0.1105499
# rgt - lon : 126.9600839 - 126.849534 = 0.1105499
lon_margin = 0.111

btm=float(lat)-lat_margin
lft=float(lon)-lon_margin
top=float(lat)+lat_margin
rgt=float(lon)+lon_margin

# 최초 요청 시 디폴트 값으로 설정되어 있으나, 원하는 값으로 구성
rletTpCds="SG" #상가
tradTpCds="A1:B1:B2" #매매/전세/월세 매물 확인


IndexError: list index out of range

최초 요청 메시지에서 각 속성값 확인 가능

_tradTpCd = [{tagCd: 'A1', uiTagNm: '매매'}, 
             {tagCd: 'B1', uiTagNm: '전세'},  
             {tagCd: 'B2', uiTagNm: '월세'},
             {tagCd: 'B3', uiTagNm: '단기임대'}];
_rletTpCd = [{tagCd: 'APT', uiTagNm: '아파트'}, {tagCd: 'OPST', uiTagNm: '오피스텔'}, {tagCd: 'VL', uiTagNm: '빌라'},
             {tagCd: 'ABYG', uiTagNm: '아파트분양권'}, {tagCd: 'OBYG', uiTagNm: '오피스텔분양권'}, {tagCd: 'JGC', uiTagNm: '재건축'},
             {tagCd: 'JWJT', uiTagNm: '전원주택'}, {tagCd: 'DDDGG', uiTagNm: '단독/다가구'}, {tagCd: 'SGJT', uiTagNm: '상가주택'},
             {tagCd: 'HOJT', uiTagNm: '한옥주택'}, {tagCd: 'JGB', uiTagNm: '재개발'}, {tagCd: 'OR', uiTagNm: '원룸'},
             {tagCd: 'GSW', uiTagNm: '고시원'}, {tagCd: 'SG', uiTagNm: '상가'}, {tagCd: 'SMS', uiTagNm: '사무실'},
             {tagCd: 'GJCG', uiTagNm: '공장/창고'}, {tagCd: 'GM', uiTagNm: '건물'}, {tagCd: 'TJ', uiTagNm: '토지'},
             {tagCd: 'APTHGJ', uiTagNm: '지식산업센터'}];

In [ ]:
# clusterList?view 를 통한 그룹(단지)의 데이터를 가져온다.
remaked_URL = "https://m.land.naver.com/cluster/clusterList?view=atcl&cortarNo={}&rletTpCd={}&tradTpCd={}&z={}&lat={}&lon={}&btm={}&lft={}&top={}&rgt={}"\
     .format(cortarNo, rletTpCds, tradTpCds, z, lat, lon,btm,lft,top,rgt)

res2 = requests.get(remaked_URL)
json_str = json.loads(json.dumps(res2.json()))

In [ ]:
values = json_str['data']['ARTICLE']

# 큰 원으로 구성되어 있는 전체 매물그룹(values)을 load 하여 한 그룹씩 세부 쿼리 진행
for v in values:
    lgeo = v['lgeo']
    count = v['count']
    z2 = v['z']
    lat2 = v['lat']
    lon2 = v['lon']

    len_pages = count / 20 + 1
    for idx in range(1, math.ceil(len_pages)):
        
        remaked_URL2 = "https://m.land.naver.com/cluster/ajax/articleList?""itemId={}&mapKey=&lgeo={}&showR0=&" \
               "rletTpCd={}&tradTpCd={}&z={}&lat={}&""lon={}&totCnt={}&cortarNo={}&page={}"\
            .format(lgeo, lgeo, rletTpCds, tradTpCds, z2, lat2, lon2, count,cortarNo, idx)


In [ ]:
atclNo = v['atclNo']        # 물건번호
rletTpNm = v['rletTpNm']    # 상가구분
tradTpNm = v['tradTpNm']    # 매매/전세/월세 구분
prc = v['prc']              # 가격
spc1 = v['spc1']            # 계약면적(m2) -> 평으로 계산 : * 0.3025
spc2 = v['spc2']            # 전용면적(m2) -> 평으로 계산 : * 0.3025
hanPrc = v['hanPrc']        # 보증금                
rentPrc = v['rentPrc']      # 월세
flrInfo = v['flrInfo']      # 층수(물건층/전체층)
tagList = v['tagList']      # 기타 정보
rltrNm = v['rltrNm']        # 부동산
detaild_information = "https://m.land.naver.com/article/info/{}".format(atclNo)